<a href="https://colab.research.google.com/github/lizagonch/NLP/blob/master/sem12_active_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Active Learning

Активное обучение $-$ класс алгоритмов обучения моделей машинного обучения. Алгоритмы активного обучения отличаются тем, могут интерактивно запрашивать пользователя (или некоторый другой источник информации) для разметки новых примеров данных.

<img src="active_learning.png">

## Active Learning Strategies

#### Pool-Based Sampling

В этом сценарии экземпляры извлекаются из всего пула данных и им присваивается информативная оценка, которая показывает, насколько хорошо текущий алгоритм «понимает» данные. 

Затем система выбираются и размечаются наиболее информативные примеры.

#### Uncertainty sampling


В рамках этого алгоритма размечаются те примеры, на которых текущая модель наименее уверена.

В качестве функций "уверенности" можно использовать вероятности классов или расстояния до разделяющей гиперплоскости.

#### Membership Query Synthesis

Здесь алгритм обучения модели генерирует свои собственные примеры из некоторого настраиваемого распределения. 
Эти сгенерированные примеры отправляются на разметку и модель дообучается с учетом разметки этих примеров.

#### Query by Committee

Идея: построить ансамбль моделей $a_1,...,a_T$. 

Выбирать новые объекты $x_i$ с наибольшей несогласованностью решений ансамбля моделей.



Принцип максимума энтропии: выбираем $x_i$, на котором $a_t(x_i)$ максимально различны.

Принцип максимума средней $KL$-дивергенции:выбираем $x_i$ , на котором $P_t(y|x_i)$ максимально различны:

$С(y|u) = \frac{1}{T}\sum_{t=1}^T P_t(y|u)$ - консенсус комитета 

## SVM для Active Learning

Некоторые активные алгоритмы обучения построены на алгоритме SVM и используют структуру SVM для определения того, какие точки данных нужно размечать. 

SVM используется для определения уверенности модели в предсказании на каждом из примеров выборки. 
В качестве меры уверенности служит расстояние от объекта до построенной не текущей итерации разделяющей гиперплоскости.

## Active Learning for texts classification

Рассмотрим алгоритм pool-based active learning на примере задачи классификации твитов по тональности.


1. Разделить данные на X_pool (выборка, которую можно размечать) и X_test.
2. Выбрать $k$ примеров из X_pool для начального X_train и разметить их. Остальные данные в X_pool $-$ валидационное множество. 
3. Обучить модель на  X_train.
5. Сделать predict обученной моделью на X_pool, вычислить вероятности для каждого $x_i$.
6. Вычислить качество работы модели на X_test.
7. Выбрать $k$ наиболее информативных объектов из X_pool, основываясь на уверенности модели в каждом из объектов (например, вероятности классов).
8. Переменести эти $k$ выбранных объектов в X_train.
9. Если качество работы модели на X_test достаточное, то останавливаемся, иначе возвращаемся к шагу 3.

In [0]:
from sklearn.metrics import f1_score

In [0]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from nltk import ngrams

from sklearn.linear_model import LogisticRegression 
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
! wget https://raw.githubusercontent.com/PragmaticsLab/NLP-course-AMI/dev/seminars/sem12_active_learning/spam_text_classification_data.csv

--2019-11-27 05:07:36--  https://raw.githubusercontent.com/PragmaticsLab/NLP-course-AMI/dev/seminars/sem12_active_learning/spam_text_classification_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 485702 (474K) [text/plain]
Saving to: ‘spam_text_classification_data.csv’

spam_text_classific 100%[===================>] 474.32K  --.-KB/s    in 0.06s   

2019-11-27 05:07:41 (7.61 MB/s) - ‘spam_text_classification_data.csv’ saved [485702/485702]



In [4]:
df = pd.read_csv('spam_text_classification_data.csv')
print(df.shape)
df.head()

(5572, 2)


,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [0]:
df['label'] = [0 if category == 'ham' else 1 for category in df['Category']]

In [0]:
X,X_test,y,y_test = train_test_split(np.array(df['Message']), np.array(df['label']))

In [0]:
def get_confidence(class_probs):
    return abs(0.5-class_probs[0])

In [0]:
train_size = 50

dataset_size = X.shape[0]
target_score = 0.95
score = 0
step = 10

X_train = X[:train_size]
y_train = y[:train_size]
X_pool = X[train_size:]
y_pool = y[train_size:]

scores = [0]
train_szs = [0]

while score < target_score and train_size <= dataset_size:
    vec = CountVectorizer(ngram_range=(1, 1))
    bow = vec.fit_transform(X_train)
    clf = LogisticRegression()
    clf = clf.fit(bow,y_train)
    pred = clf.predict(vec.transform(X_test))
    
    print("{0} train samples".format(train_size))
    print(classification_report(pred, y_test))
    score = f1_score(pred, y_test)
    scores.append(score)
    train_szs.append(train_size)
    
    pred_probs = clf.predict_proba(vec.transform(X_pool))
    confidences = [get_confidence(probs) for probs in pred_probs]
    
    X_train = np.concatenate([X_train, X_pool[np.argsort(confidences)[:step]]])
    y_train = np.concatenate([y_train, y_pool[np.argsort(confidences)[:step]]])
    X_pool = X_pool[sorted(np.argsort(confidences)[step:])]
    y_pool = y_pool[sorted(np.argsort(confidences)[step:])]
    train_size += step

In [22]:
train_size = 50

dataset_size = X.shape[0]
target_score = 0.95
score = 0
step = 10

X_train = X[:train_size]
y_train = y[:train_size]
X_pool = X[train_size:]
y_pool = y[train_size:]

scores = [0]
train_szs = [0]

while score < target_score and train_size <= dataset_size:
    vec = CountVectorizer(ngram_range=(1, 1))
    bow = vec.fit_transform(X_train)
    clf = LogisticRegression()
    clf = clf.fit(bow,y_train)
    pred = clf.predict(vec.transform(X_test))
    
    print("{0} train samples".format(train_size))
    print(classification_report(pred, y_test))
    score = f1_score(pred, y_test)
    scores.append(score)
    train_szs.append(train_size)
    
    pred_probs = clf.predict_proba(vec.transform(X_pool))
    #confidences = [get_confidence(probs) for probs in pred_probs]
    e = (-pred_probs * np.log2(pred_probs)).sum(axis=1)
    print (e)
    
    X_train = np.concatenate([X_train, X_pool[np.argsort(e)[::-1][:step]]])
    y_train = np.concatenate([y_train, y_pool[np.argsort(e)[::-1][:step]]])
    X_pool = X_pool[sorted(np.argsort(e)[::-1][step:])]
    y_pool = y_pool[sorted(np.argsort(e)[::-1][step:])]
    train_size += step

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


50 train samples
              precision    recall  f1-score   support

           0       1.00      0.88      0.93      1392
           1       0.01      1.00      0.01         1

    accuracy                           0.88      1393
   macro avg       0.50      0.94      0.47      1393
weighted avg       1.00      0.88      0.93      1393

[0.1581867  0.46981259 0.37086783 ... 0.45746971 0.50768414 0.43528067]
60 train samples
              precision    recall  f1-score   support

           0       1.00      0.88      0.94      1378
           1       0.09      1.00      0.16        15

    accuracy                           0.89      1393
   macro avg       0.54      0.94      0.55      1393
weighted avg       0.99      0.89      0.93      1393

[0.15410785 0.48566504 0.36322672 ... 0.45717283 0.54895692 0.5198989 ]
70 train samples
              precision    recall  f1-score   support

           0       1.00      0.90      0.95      1344
           1       0.27      0.96      0.4

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


100 train samples
              precision    recall  f1-score   support

           0       0.99      0.95      0.97      1266
           1       0.65      0.89      0.75       127

    accuracy                           0.95      1393
   macro avg       0.82      0.92      0.86      1393
weighted avg       0.96      0.95      0.95      1393

[0.09863605 0.56306073 0.39684579 ... 0.55531347 0.76725327 0.8118951 ]
110 train samples
              precision    recall  f1-score   support

           0       0.99      0.96      0.97      1252
           1       0.71      0.88      0.78       141

    accuracy                           0.95      1393
   macro avg       0.85      0.92      0.88      1393
weighted avg       0.96      0.95      0.95      1393

[0.08389759 0.5289699  0.39953238 ... 0.48654647 0.75506665 0.86686542]
120 train samples
              precision    recall  f1-score   support

           0       0.99      0.96      0.98      1251
           1       0.73      0.90      

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


...

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


4130 train samples
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1241
           1       0.85      0.97      0.91       152

    accuracy                           0.98      1393
   macro avg       0.92      0.98      0.95      1393
weighted avg       0.98      0.98      0.98      1393

[1.29742533e-05 1.20141299e-04 7.19083807e-09 1.08385423e-05
 3.26147166e-04 6.20024445e-07 2.57839613e-04 4.01563840e-04
 3.61388488e-04 5.82699009e-05 9.59061569e-05 4.09291622e-05
 7.19083807e-09 1.81913542e-04 7.32618688e-07 6.10589406e-05
 3.48382412e-04 2.08759671e-04 3.98086175e-17 2.41462807e-04
 2.21561041e-04 2.82775048e-04 1.62310062e-04 2.15705731e-04
 2.86157449e-06 1.21654525e-05 2.09086787e-04 5.50165723e-04
 5.59692445e-05 3.60180944e-04 2.15705731e-04 4.98187197e-04
 2.75544799e-04 2.07507274e-04 1.74410685e-04 4.07241048e-04
 4.73360767e-04 3.03307206e-04 7.15238724e-05 3.06260942e-06
 5.59318772e-04 5.84291903e-06 1.70180635e-0

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


4150 train samples
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1241
           1       0.85      0.97      0.91       152

    accuracy                           0.98      1393
   macro avg       0.92      0.98      0.95      1393
weighted avg       0.98      0.98      0.98      1393

[1.29761015e-05 1.20129004e-04 7.19529202e-09 1.08398100e-05
 6.21121028e-07 5.82760227e-05 9.58046119e-05 4.09343862e-05
 7.19529202e-09 1.81892207e-04 7.32117357e-07 6.10684395e-05
 2.08778607e-04 3.98390264e-17 1.62299546e-04 2.86239357e-06
 1.21607567e-05 2.09147744e-04 5.59832955e-05 2.15813886e-04
 2.07504217e-04 1.74436138e-04 7.15408023e-05 3.05606434e-06
 5.84065599e-06 1.70211009e-05 3.95006164e-05 2.07592152e-04
 1.58228599e-04]
4160 train samples
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1241
           1       0.85      0.97      0.91       152

    accuracy               

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [40]:
train_size = 50

dataset_size = X.shape[0]
target_score = 0.95
score = 0
step = 10

X_train = X[:train_size]
y_train = y[:train_size]
X_pool = X[train_size:]
y_pool = y[train_size:]

scores = [0]
train_szs = [0]

while score < target_score and train_size <= dataset_size:
    vec = CountVectorizer(ngram_range=(1, 1))
    bow = vec.fit_transform(X_train)
    clf = LogisticRegression()
    clf = clf.fit(bow,y_train)
    pred = clf.predict(vec.transform(X_test))
    
    print("{0} train samples".format(train_size))
    print(classification_report(pred, y_test))
    score = f1_score(pred, y_test)
    scores.append(score)
    train_szs.append(train_size)
    
    pred_probs = clf.predict_proba(vec.transform(X_pool))
    #print(pred_probs)
    #confidences = [get_confidence(probs) for probs in pred_probs]
    probs_sort = np.sort(pred_probs, axis=1)[:, ::-1]
    
    confidences = probs_sort[:, 0] - probs_sort[:, 1]
    
    X_train = np.concatenate([X_train, X_pool[np.argsort(confidences)[:step]]])
    y_train = np.concatenate([y_train, y_pool[np.argsort(confidences)[:step]]])
    X_pool = X_pool[np.argsort(confidences)[:step]]
    y_pool = y_pool[np.argsort(confidences)[:step]]
    train_size += step


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

50 train samples
              precision    recall  f1-score   support

           0       1.00      0.88      0.93      1392
           1       0.01      1.00      0.01         1

    accuracy                           0.88      1393
   macro avg       0.50      0.94      0.47      1393
weighted avg       1.00      0.88      0.93      1393

60 train samples
              precision    recall  f1-score   support

           0       1.00      0.88      0.94      1378
           1       0.09      1.00      0.16        15

    accuracy                           0.89      1393
   macro avg       0.54      0.94      0.55      1393
weighted avg       0.99      0.89      0.93      1393

70 train samples
              precision    recall  f1-score   support

           0       1.00      0.89      0.94      1375
           1       0.10      1.00      0.19        18

    accuracy                           0.89      1393
   macro avg       0.55      0.94      0.56      1393
weighted avg       0.99

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

100 train samples
              precision    recall  f1-score   support

           0       1.00      0.89      0.94      1374
           1       0.11      1.00      0.20        19

    accuracy                           0.89      1393
   macro avg       0.55      0.94      0.57      1393
weighted avg       0.99      0.89      0.93      1393

110 train samples
              precision    recall  f1-score   support

           0       1.00      0.89      0.94      1374
           1       0.11      1.00      0.20        19

    accuracy                           0.89      1393
   macro avg       0.55      0.94      0.57      1393
weighted avg       0.99      0.89      0.93      1393

120 train samples
              precision    recall  f1-score   support

           0       1.00      0.89      0.94      1374
           1       0.11      1.00      0.20        19

    accuracy                           0.89      1393
   macro avg       0.55      0.94      0.57      1393
weighted avg       0

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

...

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


4140 train samples
              precision    recall  f1-score   support

           0       1.00      0.90      0.95      1351
           1       0.22      0.90      0.35        42

    accuracy                           0.90      1393
   macro avg       0.61      0.90      0.65      1393
weighted avg       0.97      0.90      0.93      1393

4150 train samples
              precision    recall  f1-score   support

           0       1.00      0.90      0.95      1351
           1       0.22      0.90      0.35        42

    accuracy                           0.90      1393
   macro avg       0.61      0.90      0.65      1393
weighted avg       0.97      0.90      0.93      1393



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


4160 train samples
              precision    recall  f1-score   support

           0       1.00      0.90      0.95      1351
           1       0.22      0.90      0.35        42

    accuracy                           0.90      1393
   macro avg       0.61      0.90      0.65      1393
weighted avg       0.97      0.90      0.93      1393

4170 train samples
              precision    recall  f1-score   support

           0       1.00      0.90      0.95      1351
           1       0.22      0.90      0.35        42

    accuracy                           0.90      1393
   macro avg       0.61      0.90      0.65      1393
weighted avg       0.97      0.90      0.93      1393



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [33]:
vec = CountVectorizer(ngram_range=(1, 1))
bow = vec.fit_transform(X)
clf = clf.fit(bow,y)
pred = clf.predict(vec.transform(X_test))

print("{0} train samples".format(dataset_size))
print(classification_report(pred, y_test))

4179 train samples
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1241
           1       0.85      0.97      0.91       152

    accuracy                           0.98      1393
   macro avg       0.92      0.98      0.95      1393
weighted avg       0.98      0.98      0.98      1393



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [0]:
from matplotlib import pyplot as plt

%matplotlib inline

In [35]:
plt.plot(train_szs,scores)
plt.plot(train_szs, [0.95 for sz in train_szs])

Можно видеть, что для достижения лучшего качества на этом датасете дсотаточно обучиться на 300 правильно выбранных примерах.